In [24]:
import numpy as np
import pandas as pd
import os
import random
random.seed(675)

In [25]:
ejemplo=pd.read_csv("sample_submission.csv")

In [26]:
ejemplo

,Id,Predicted
0,0,NECKLACES
1,1,MICROPHONES
2,2,SMARTWATCHES
3,3,MOTORCYCLE_HELMETS
4,4,PANTS
...,...,...
71456,71456,DIECAST_VEHICLES
71457,71457,RAM_MEMORY_MODULES
71458,71458,COMPUTER_PROCESSORS
71459,71459,MOTORCYCLE_HELMETS


In [27]:
train2=pd.read_csv("train.csv")
train_original= train2
test=pd.read_csv("x_test.csv")

In [28]:
train2

,title,category
0,Bikini Malla Lunares Colores Colección Verano...,WOMEN_SWIMWEAR
1,Kit 4 Bujias Ngk Ford F-100 3.0 Bp5efs,SPARK_PLUGS
2,Campera Rompeviento adidas Originals Hard Shel...,JACKETS_AND_COATS
3,Vulk - Danner Sblk/sg91 Polarized,SUNGLASSES
4,Calza Frizada Termica,PANTS
...,...,...
107186,Memoria Ddr4 Kingston 16gb 2400 Cl15 Hyperx ...,RAM_MEMORY_MODULES
107187,Mallas Talles Grandes Importadas,WOMEN_SWIMWEAR
107188,Kit 6 Bujias Originales Bmw X 5 4.6 Is 4x4 E53...,SPARK_PLUGS
107189,Reloj Casio Unisex Retro Vintage Metal A159w A...,WRISTWATCHES


In [29]:
test

,Id,title,Predicted
0,0,Collar Marga Black,NaN
1,1,Microfono Condenser Rode Nt5,NaN
2,2,Apple Watch + Nike 42mm Silver Aluminum Case P...,NaN
3,3,Casco Abierto Shiro Sh-62 Monocolor Gris - Um,NaN
4,4,Jeans Negro Roto Chupin Hombre Ybr Jeans,NaN
...,...,...,...
71456,71456,Hot Wheels 15 Ford F150 Pickup 2015 # 119 So...,NaN
71457,71457,Memoria So-dimm Ddr3 2gb 1600mhz Pc3-12800s,NaN
71458,71458,Microprocesador Intel I7 2.93 Mhz Socket 1366,NaN
71459,71459,Jv Casco Ls2 Convertible Rebatible Ff 399 Val...,NaN


In [30]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
data= train2
le = preprocessing.LabelEncoder()
le.fit(data.category)
le.classes_
train2['category'] = le.transform(data.category)

In [31]:
train=pd.concat([train2,test],axis=0, sort=False, ignore_index=True).drop(['Id','Predicted'], axis=1)

In [32]:
train2

,title,category
0,Bikini Malla Lunares Colores Colección Verano...,48
1,Kit 4 Bujias Ngk Ford F-100 3.0 Bp5efs,41
2,Campera Rompeviento adidas Originals Hard Shel...,23
3,Vulk - Danner Sblk/sg91 Polarized,42
4,Calza Frizada Termica,33
...,...,...
107186,Memoria Ddr4 Kingston 16gb 2400 Cl15 Hyperx ...,37
107187,Mallas Talles Grandes Importadas,48
107188,Kit 6 Bujias Originales Bmw X 5 4.6 Is 4x4 E53...,41
107189,Reloj Casio Unisex Retro Vintage Metal A159w A...,49


In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import unicodedata

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
norm = lambda w : unicodedata.normalize('NFKD', w).encode('ASCII', 'ignore').decode('ASCII')
all_stopw = set()
for corpus in [ 'english','spanish']:
    all_stopw.update(set(map(norm, stopwords.words(corpus))))
for i in range(65, 123):
    all_stopw.add(chr(i))    
###conjunto de todas las stop words en 2 idiomas y el abecedario
    


In [35]:
def normalize(curr):
    # remove accent
    curr = curr.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # to lower case
    curr = curr.str.lower()
    # remove not alphanumerics or . ,
    curr = curr.str.replace('[^a-zA-Z0-9.,]', ' ') #separa caracteres alfanuméricos por - / etc
   
    # let , and . be the same char
    curr = curr.str.replace('[.]', ' ') #Se pasan los puntos a espacio
   
    # remove . , not between numbers
    curr = curr.str.replace('(?<=[0-9])[,]+(?=[0-9])', ' ')
    curr = curr.str.replace('[,]', ' ')
   
    # set all digits to 0
    curr = curr.str.replace('[0-9]', ' ')
   
    # separate ' <digits><letters ' like in 22g or 12ms
    curr = curr.str.replace('(^| )([0-9]+)([a-zA-Z]+)($| )', r'\1\2 \3\4')
   
    # Corrige cuando hay más de 4 letras iguales consecutivas reemplazandolas por sólo una
    curr = curr.str.replace(r'([a-zA-Z])\1{3,}', r'\1') # 3 es 4  
   
    return curr

In [36]:
X_data = train.title
X_full = normalize(X_data)
sp2 = len(train)
xremo = lambda s : ' '.join([w for w in s if w not in all_stopw])
X_full = X_full.str.split().apply(xremo)


from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(strip_accents='ascii', binary = True, min_df= 2, max_df = .95,
#                              analyzer = 'char',
                             ngram_range=(1,2),
                            )
X_train_counts = count_vect.fit_transform(X_full)
print(X_train_counts.shape, X_train_counts.count_nonzero())


from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(norm='l2', use_idf=False, smooth_idf=True, sublinear_tf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_go = X_train_tfidf
print(X_train_tfidf.shape)
x_train, x_test = X_go[0:train2.shape[0]], X_go[train2.shape[0]:]
y_train= train2['category']

(178652, 120374) 1692145
(178652, 120374)


Probamos muchas combinaciones posibles de ngrams, métricas de regularización y otros parametros, conclusion es mejor con n=2 n gramas

In [37]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import Perceptron

clf_ovr = OneVsRestClassifier(Perceptron()).fit(x_train, y_train)
clf = clf_ovr


prediction_val = clf.predict(x_test)
prediction_val.shape

(71461,)

In [38]:
resultado= pd.DataFrame(data=prediction_val)
resultado2 = resultado.rename_axis('Id').reset_index()
resultado2['Predicted']=le.inverse_transform(resultado2[0])
resultado2= resultado2.drop([0], axis=1)
resultado2.to_csv(r'C:\Users\Mario\python\machine_learning\clasificador_mercado_libre\estesi\sample_submission.csv', index = False)

In [39]:
resultado2

,Id,Predicted
0,0,NECKLACES
1,1,MICROPHONES
2,2,SMARTWATCHES
3,3,MOTORCYCLE_HELMETS
4,4,PANTS
...,...,...
71456,71456,DIECAST_VEHICLES
71457,71457,RAM_MEMORY_MODULES
71458,71458,COMPUTER_PROCESSORS
71459,71459,MOTORCYCLE_HELMETS


Ejemplos de predicción

In [40]:
resultado2.iloc[55]

Id               55
Predicted    SHIRTS
Name: 55, dtype: object

In [41]:
test.iloc[55]

Id                                                          55
title        Camisa Bebe Escocesa Manga Larga Pitocatalan (...
Predicted                                                  NaN
Name: 55, dtype: object

In [42]:
## frecuencia palabras
words=dict()
for w in X_full:
    palabras=w.split()
    for j in palabras:
        words[j]=words.get(j,0)+1
words


{'bikini': 1270,
 'malla': 1806,
 'lunares': 67,
 'colores': 1199,
 'coleccion': 1234,
 'verano': 491,
 'kit': 4825,
 'bujias': 1521,
 'ngk': 1407,
 'ford': 1736,
 'bp': 86,
 'efs': 14,
 'campera': 3357,
 'rompeviento': 158,
 'adidas': 2199,
 'originals': 233,
 'hard': 36,
 'shell': 71,
 'jkt': 30,
 'hombr': 9,
 'gr': 179,
 'vulk': 156,
 'danner': 1,
 'sblk': 14,
 'sg': 257,
 'polarized': 52,
 'calza': 328,
 'frizada': 46,
 'termica': 279,
 'reloj': 6290,
 'samsung': 3927,
 'sm': 239,
 'smr': 5,
 'smartwatch': 1290,
 'gps': 601,
 'cardio': 81,
 'bluetooth': 1423,
 'labiales': 741,
 'kylie': 521,
 'jenner': 140,
 'combo': 896,
 'matte': 929,
 'revista': 1465,
 'susana': 42,
 'gimenez': 16,
 'armamec': 8,
 'piezas': 1362,
 'concept': 9,
 'tipo': 1240,
 'mustang': 153,
 'lego': 1723,
 'oferta': 2433,
 'casco': 3694,
 'moto': 1115,
 'momo': 28,
 'classic': 697,
 'blanco': 1750,
 'negro': 3052,
 'ml': 361,
 'lavalle': 51,
 'motos': 596,
 'monitor': 2944,
 'led': 8739,
 'full': 1299,
 'hd': 